In [1]:
import numpy as np
import scipy.io as sio
import os
import sys
import math
from scipy import linalg,signal
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC 
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import normalize

asd_data = sio.loadmat('whole_asd_fmri.mat')
nt_data = sio.loadmat('whole_tdc_fmri.mat')

def normalise_matrix (matrix1, matrix2, matrix3 ):
	for l in range (0,90):
		a11 = np.trim_zeros(matrix2[l])
		a11_len = len(a11)
		for m in range (0,a11_len): 
			a11[m] = a11[m] - matrix3[l]
		u = 500 - a11_len
		a11 = np.pad(a11,(0,u),'constant',constant_values=0)
		a11 = np.asarray(a11)
		matrix1[l] = a11

	
def mean_row(matrix1):
	r = np.zeros([90,1])
	for i in range(0,90):
		r[i] = np.mean(np.trim_zeros(matrix1[i]))
	r=np.reshape(r,(90,1))
	return r

na = 443
nn = 435
x = asd_data['x']
x = np.transpose(x,(2,1,0))

y = nt_data['y']
y = np.transpose(y,(2,1,0))


# autistic and neurotypical arrays
a = np.ndarray(shape = (438,) , dtype="object")
n = np.ndarray(shape = (434,) , dtype="object")

#mean of each row of the matrices use axis=1
ma = np.ndarray(shape = (438,90,1) , dtype="object")
mn = np.ndarray(shape = (434,90,1) , dtype="object")

#normalised input matrices
xa = np.ndarray(shape = (438,) , dtype="object")
xn = np.ndarray(shape = (434,) , dtype="object")

cnt1=0
for i in range(0,na): 
	if((i != 6) and (i != 7) and (i != 10) and (i!= 325) and (i!= 330) ):
		a[cnt1] = x[i]
		cnt1 = cnt1+1
		

cnt2=0
for j in range(0,nn):
	if(( j != 62)):
		n[cnt2] = y[j]
		cnt2 = cnt2+1

na=438
nn=434
for i in range(0, na):
	xa[i] = np.zeros((90,500))

for j in range(0, nn):
	xn[j] = np.zeros((90,500))

#finding mean of every row in 2d matrix for each subject

for i in range(0,na):
	ma[i] = mean_row(a[i])

for j in range(0,nn):
	mn[j] = mean_row(n[j])

#getting normalised matrix

for i in range(0,na):
	normalise_matrix(xa[i],a[i],ma[i])
	
for j in range(0,nn):
	normalise_matrix(xn[j],n[j],mn[j])


/usr/local/lib/python2.7/dist-packages/sklearn/utils/__init__.py:9: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .murmurhash import murmurhash3_32
/usr/local/lib/python2.7/dist-packages/sklearn/utils/extmath.py:24: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._logistic_sigmoid import _log_logistic_sigmoid
/usr/local/lib/python2.7/dist-packages/sklearn/utils/extmath.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .sparsefuncs_fast import csr_row_norms
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/cluster/supervised.py:23: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .expected_mutual_info_fast import expected_mutual_information
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/pairwise.py:30: RuntimeWarning: numpy.dtype size chan

In [2]:
no_train = 786
no_test = (na+nn) - no_train
xta = int(no_train/2) + 2       #329
xtn = no_train - xta            #326

np.random.shuffle(xa)   #shuffles the matrix and saves in itself, returns nothing
np.random.shuffle(xn)

xa_train = np.ndarray(shape = (xta,90,500) , dtype="object")
xa_test = np.ndarray(shape = ((na-xta),90,500) , dtype="object")
xn_train = np.ndarray(shape = (xtn,90,500) , dtype="object")
xn_test = np.ndarray(shape = ((nn-xtn),90,500) , dtype="object")

for i in range(0,xta):
	xa_train[i] = xa[i]
for i in range(0,xtn):
	xn_train[i] = xn[i]
for i in range(0,na-xta):
	xa_test[i] = xa[xta+i]
for i in range(0,nn-xtn):
	xn_test[i] = xn[xtn+i]

#finding connectivity matrix
c_xa = np.ndarray(shape = (xta,90,90) , dtype="object")
c_xn = np.ndarray(shape = (xtn,90,90) , dtype="object")

#transpose of xa and xn
xa_t = np.ndarray(shape = (xta,500,90),dtype="object")
xn_t = np.ndarray(shape = (xtn,500,90),dtype="object")

#total mean corrected input
X_train = np.ndarray(shape = (no_train,),dtype="object")  #(872,90,500)
X_test = np.ndarray(shape = (no_test,),dtype="object")  #(872,90,500)
Y_train  = np.ndarray(shape = (no_train,) , dtype="object")  #(872,90,500)
Y_test  = np.ndarray(shape = (no_test,) , dtype="object")  #(872,90,500)

for i in range (0,xta):
	X_train[i] = xa_train[i]
for j in range (0,xtn):
	X_train[j + xta] = xn_train[j]


for i in range (0,na-xta):
	X_test[i] = xa_test[i]
for j in range (0,nn-xtn):
	X_test[(na-xta)+j] = xn_test[j]

    
xa_train = xa_train.astype(float)
xn_train = xn_train.astype(float)
xa_t = xa_t.astype(float)
xn_t = xn_t.astype(float)
for i in range (0, xta):
	xa_t[i] = xa_train[i].transpose()
	c_xa[i] = np.matmul(xa_train[i],xa_t[i])

for j in range (0, xtn):
	xn_t[j] = xn_train[j].transpose()
	c_xn[j] = np.matmul(xn_train[j],xn_t[j])


#trace of c_xa and c_xn
tr_cxa = np.zeros((xta,1))
tr_cxn = np.zeros((xtn,1))


for i in range(0, xta):
	tr_cxa[i] = np.trace(c_xa[i])

for j in range(0, xtn):
	tr_cxn[j] = np.trace(c_xn[j])

for i in range (0,xta):
	c_xa[i] = np.divide(c_xa[i], tr_cxa[i])

for j in range (0,xtn):
	c_xn[j] = np.divide(c_xn[j], tr_cxn[j])

#finding mean connectivity matrices for asd and neurotypical
m_cxa = np.zeros((90,90))
m_cxn = np.zeros((90,90))

for i in range (0, xta):
	for j in range (0,90):
		for k in range (0,90):
			m_cxa[j][k] = m_cxa[j][k] + c_xa[i][j][k]

for i in range (0, xtn):
	for j in range (0,90):
		for k in range (0,90):
			m_cxn[j][k] = m_cxn[j][k] + c_xn[i][j][k]

Co = np.add (m_cxa , m_cxn)
Co = np.divide(Co , no_train)

m_cxa = np.divide(m_cxa , xta)
m_cxn = np.divide(m_cxn , xtn)

#Finding the SVD of the connectivity matrix
#Bo,xo,Bo_tr = np.linalg.svd(Co, compute_uv=True)
#Reshaping the eigen value matrix
xo_notsort, Bo_notsort = np.linalg.eig(Co)
CSP1 = []
xo = sorted(xo_notsort, reverse = True)
for i in xo:
    CSP1.append(Bo_notsort[xo_notsort==i][0])
Bo = np.asarray(CSP1)

Xo=np.diag(xo)
inverse_Xo = np.linalg.inv(Xo)
sqrt_inv = np.sqrt(inverse_Xo)
BoT = np.transpose(Bo)

#whitening Matrix
W = np.matmul(sqrt_inv,BoT)  #(90,90)
Wt = np.transpose(W)

var_1 = np.matmul(W , m_cxa)
m_cxa_bar = np.matmul(var_1 , Wt)   #(90,90)
m_cxa_bar_t = np.transpose(m_cxa_bar)

#v= np.array_equal(m_cxa_bar, m_cxa_bar_t)
#print(v)

var_2 = np.matmul(W , m_cxn)
m_cxn_bar = np.matmul(var_2 , Wt) #(90,90)
m_cxn_bar_t = np.transpose(m_cxn_bar)


CSP = []
B_notsort , U = linalg.eig(m_cxa_bar, b=m_cxn_bar)
B = sorted(B_notsort, reverse = True)
for i in B:
    CSP.append(U[B_notsort==i][0])
CSP_arr = np.asarray(CSP)

U_t = np.transpose(CSP_arr)
P = np.matmul(U_t,W)
P_inv = np.linalg.inv(P)
P = P.astype(float)
P_inv = P_inv.astype(float)

for i in range(0,no_train):
	Y_train[i] = np.matmul(P,X_train[i].astype(float))
for i in range(0,no_test):
	Y_test[i] = np.matmul(P,X_test[i].astype(float))

no_feat = 20 # each for mean and max corr
tb =int(no_feat/2)
nf2 = 2*no_feat

f_train_mean = np.ndarray(shape =(no_train,no_feat,1) , dtype = "object")
f_train_maxcorr = np.ndarray(shape =(no_train,no_feat,1) , dtype = "object")
f_test_mean = np.ndarray(shape = (no_test,no_feat,1) , dtype="object")
f_test_maxcorr = np.ndarray(shape = (no_test,no_feat,1) , dtype="object")
f_train = np.ndarray(shape =(no_train,(nf2),1) , dtype = "object")
f_test = np.ndarray(shape =(no_train,(nf2),1) , dtype = "object")

for k in range (0,no_train):
    for i in range(0,tb):
        f_train_mean[k][i] = np.mean(np.trim_zeros(Y_train[k][i]))
        f_train_maxcorr[k][i] = np.amax(signal.correlate(np.trim_zeros(Y_train[k][i]),np.trim_zeros(Y_train[k][i])))
    cnt5 = tb
    for j in range((90-tb),90):
        f_train_mean[k][cnt5] = np.mean(np.trim_zeros(Y_train[k][j]))
        f_train_maxcorr[k][cnt5] = np.amax(signal.correlate(np.trim_zeros(Y_train[k][j]),np.trim_zeros(Y_train[k][j])))
        cnt5 = cnt5+1
        
for k in range (0,no_test):
    for i in range(0,tb):
        f_test_mean[k][i] = np.mean(np.trim_zeros(Y_test[k][i]))
        f_test_maxcorr[k][i] = np.amax(signal.correlate(np.trim_zeros(Y_test[k][i]),np.trim_zeros(Y_test[k][i])))
    cnt6 = tb
    for j in range((90-tb),90):
        f_test_mean[k][cnt6] = np.mean(np.trim_zeros(Y_test[k][j]))
        f_test_maxcorr[k][cnt6] = np.amax(signal.correlate(np.trim_zeros(Y_test[k][j]),np.trim_zeros(Y_test[k][j])))
        cnt6 = cnt6+1

for k in range (0,no_train):
	for i in range (0, no_feat):
		f_train[k][i] = f_train_mean[k][i]
	for j in range (no_feat,nf2):
		f_train[k][j] = f_train_maxcorr[k][j-no_feat]
for k in range (0,no_test):
	for i in range (0, no_feat):
		f_test[k][i] = f_test_mean[k][i]
	for j in range (no_feat,nf2):
		f_test[k][j] = f_test_maxcorr[k][j-no_feat]


qw = (2*no_feat)+1

fn1_train = np.zeros([no_train,nf2])
fn1_test = np.zeros([no_test,nf2])

for i in range(0,no_train):
	for j in range(0,nf2):
		fn1_train[i][j] = f_train[i][j][0]
for i in range(0,no_test):
	for j in range(0,nf2):
		fn1_test[i][j] = f_test[i][j][0]


#Normalizing the data sample wise
fn_train = normalize(fn1_train, norm='max', axis=1)
fn_test = normalize(fn1_test, norm='max', axis=1)

fn_a_train = np.zeros([xta,qw])
fn_n_train = np.zeros([xtn,qw])
fn_a_test = np.zeros([na-xta,qw])
fn_n_test = np.zeros([nn-xtn,qw])

#only top tb and bottom tb no. of features(columns) from fn are chosen
#the last column of fn_a and fn_n is set to class no: 1 if autistic and 0 if neurotypical	
for i in range(0,xta):
    for j in range(0,nf2): 
    	fn_a_train[i][j] = fn_train[i][j]
    fn_a_train[i][nf2] = 1          #setting class

for i in range(0,xtn):
    for j in range(0,nf2):
        fn_n_train[i][j] = fn_train[i+xta][j] 
    fn_n_train[i][nf2] = 0        #setting class
    
for i in range(0,na-xta):
    for j in range(0,nf2): 
    	fn_a_test[i][j] = fn_test[i][j]
    fn_a_test[i][nf2] = 1 

for i in range(0,nn-xtn):
    for j in range(0,nf2):
        fn_n_test[i][j] = fn_test[i+(na-xta)][j]
    fn_n_test[i][nf2] = 0        #setting class



Xx_train = np.ndarray(shape = (no_train,qw), dtype = object)
Xx_test = np.ndarray(shape = (no_test,qw), dtype = object)

X_train_s = np.ndarray(shape = (no_train,nf2), dtype = object)
X_test_s = np.ndarray(shape = (no_test,nf2), dtype = object)
y_test_s = np.zeros([no_test])
y_train_s = np.zeros([no_train])

for i in range(0,xta):
	Xx_train[i] = fn_a_train[i]

for j in range(0,xtn):
	Xx_train[xta+j] = fn_n_train[j]

for i in range(0,na - xta):
	Xx_test[i] = fn_a_test[i]

for j in range(0,nn - xtn):
	Xx_test[(na-xta)+j] = fn_n_test[j]


np.random.shuffle(Xx_train)
np.random.shuffle(Xx_test)

for i in range(0,no_train):
	for j in range(0,nf2):
		X_train_s[i][j] = Xx_train[i][j]
	y_train_s[i] = Xx_train[i][nf2]

for i in range(0,no_test):
	for j in range(0,nf2):
		X_test_s[i][j] = Xx_test[i][j]
	y_test_s[i] = Xx_test[i][nf2]


/usr/lib/python2.7/dist-packages/scipy/signal/signaltools.py:492: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()


In [3]:
std_dev = X_train_s.std()
gam = 1/(std_dev*nf2)

svclassifier = SVC(C=2.0,kernel='rbf',gamma = gam)
svclassifier.fit(X_train_s, y_train_s)
y_pred_s = svclassifier.predict(X_test_s)
print('y_pred',y_pred_s.shape)
print('y_test',y_test_s.shape)
score=0
score_a = 0
score_n = 0
for i in range(0,no_test):
    if(y_pred_s[i]==y_test_s[i]):
        score = score+1
        if(y_pred_s[i]==1):
            score_a = score_a+1
        if(y_pred_s[i]==0):
           score_n = score_n+1
acc = (float(score)*100)/(float(no_test))
acc_a = (float(score_a)*100) / (float(na-xta))
acc_n = (float(score_n)*100) / (float(nn-xtn))
print('accuracy = ',acc)
print('autistic class accuracy = ', acc_a)
print('neurotypical class accuracy = ',acc_n)
print('Confusion matrix \n')
print(confusion_matrix(y_test_s, y_pred_s))
print(classification_report(y_test_s,y_pred_s))


('y_pred', (86,))
('y_test', (86,))
('accuracy = ', 54.651162790697676)
('autistic class accuracy = ', 58.13953488372093)
('neurotypical class accuracy = ', 51.16279069767442)
Confusion matrix 

[[22 21]
 [18 25]]
             precision    recall  f1-score   support

        0.0       0.55      0.51      0.53        43
        1.0       0.54      0.58      0.56        43

avg / total       0.55      0.55      0.55        86

